In [47]:
import warnings
warnings.filterwarnings("ignore")

import requests
import numpy as np
import pandas as pd
import geopandas as gpd # GeoPandas library for spatial analytics

In [48]:
data = gpd.read_file("../output/film permit_city.shp" )

In [49]:
data_sel = data[['EventID','StartDateT','EndDateTim','SegmentID','geometry']].drop_duplicates()
data_sel

,EventID,StartDateT,EndDateTim,SegmentID,geometry
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"LINESTRING (1000675.780 203773.602, 1000590.62..."
2,629351,03/08/2022 07:00:00 AM,03/08/2022 10:00:00 PM,0048509,"LINESTRING (1010582.250 192025.794, 1010809.74..."
3,629319,03/08/2022 07:00:00 AM,03/08/2022 09:00:00 PM,0044431,"LINESTRING (1002265.974 195997.473, 1002202.52..."
4,629319,03/08/2022 07:00:00 AM,03/08/2022 09:00:00 PM,0044457,"LINESTRING (1002202.526 196249.592, 1002715.21..."
5,629319,03/08/2022 07:00:00 AM,03/08/2022 09:00:00 PM,0044453,"LINESTRING (1002022.559 196965.059, 1002571.01..."
...,...,...,...,...,...
44831,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0105864,"LINESTRING (982584.908 196291.744, 982718.819 ..."
44832,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023296,"LINESTRING (982183.843 196510.021, 982273.246 ..."
44833,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023304,"LINESTRING (982273.246 196628.042, 982369.951 ..."
44834,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023303,None


In [50]:
data_sel_buffer=data_sel
data_sel_buffer['geometry']= data_sel.buffer(3,cap_style=2)

In [51]:
ct = gpd.read_file("../vector/2020 Census Tract_NYC" )
ct_sel = ct[['BoroCT2020','NTAName','GEOID','geometry']]

In [52]:
ct_data = ct_sel.sjoin(data_sel_buffer, how="left")
ct_data

,BoroCT2020,NTAName,GEOID,geometry,index_right,EventID,StartDateT,EndDateTim,SegmentID
0,1000100,The Battery-Governors Island-Ellis Island-Libe...,36061000100,"MULTIPOLYGON (((972081.788 190733.467, 972184....",NaN,NaN,NaN,NaN,NaN
1,1000201,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19...",30549.0,608299.0,10/27/2021 03:00:00 PM,10/28/2021 01:00:00 AM,0024540
1,1000201,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19...",18895.0,593957.0,08/03/2021 06:00:00 AM,08/03/2021 10:00:00 PM,0024542
1,1000201,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19...",18891.0,593957.0,08/03/2021 06:00:00 AM,08/03/2021 10:00:00 PM,0024545
1,1000201,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19...",17771.0,593260.0,07/28/2021 08:00:00 AM,07/29/2021 02:00:00 AM,9003304
...,...,...,...,...,...,...,...,...,...
2322,5031902,Mariner's Harbor-Arlington-Graniteville,36085031902,"POLYGON ((939747.491 171919.373, 939778.967 17...",1821.0,559289.0,01/26/2021 07:00:00 AM,01/26/2021 10:00:00 PM,0110300
2322,5031902,Mariner's Harbor-Arlington-Graniteville,36085031902,"POLYGON ((939747.491 171919.373, 939778.967 17...",1824.0,559289.0,01/26/2021 07:00:00 AM,01/26/2021 10:00:00 PM,0005790
2322,5031902,Mariner's Harbor-Arlington-Graniteville,36085031902,"POLYGON ((939747.491 171919.373, 939778.967 17...",1825.0,559289.0,01/26/2021 07:00:00 AM,01/26/2021 10:00:00 PM,0005786
2323,5032300,Mariner's Harbor-Arlington-Graniteville,36085032300,"POLYGON ((936740.120 173125.853, 936712.951 17...",NaN,NaN,NaN,NaN,NaN


# Per Month

In [53]:
from datetime import date

In [54]:
def range_to_list(a, b):
    return pd.date_range(a, b, freq = 'H')[:-1]

In [55]:
data_sel_buffer['StartEnd'] = [[i, j] for i, j in zip( data_sel_buffer['StartDateT'], data_sel_buffer['EndDateTim'])]
data_sel_buffer['TimeList'] = data_sel_buffer['StartEnd'].apply(lambda x: range_to_list(x[0], x[1]))
data_sel_buffer['month'] = data_sel_buffer['TimeList'].apply(lambda x: list(set(x.month)))

## Only 2021

In [56]:
data_sel_buffer['StartDateTime'] = data_sel_buffer['StartDateT'].apply(pd.to_datetime)
data_sel_2021 = data_sel_buffer[data_sel_buffer['StartDateTime']<'2022-1-1']

In [57]:
data_ct_2021 = data_sel_2021.sjoin(ct_sel, how="left")
data_ct_2021

,EventID,StartDateT,EndDateTim,SegmentID,geometry,StartEnd,TimeList,month,StartDateTime,index_right,BoroCT2020,NTAName,GEOID
214,559924,01/30/2021 06:00:00 AM,01/30/2021 09:00:00 PM,0116901,"POLYGON ((1032542.597 227709.246, 1032507.851 ...","[01/30/2021 06:00:00 AM, 01/30/2021 09:00:00 PM]","DatetimeIndex(['2021-01-30 06:00:00', '2021-01...",[1],2021-01-30 06:00:00,2070.0,4094500,College Point,36081094500
220,559658,01/28/2021 10:00:00 AM,01/29/2021 01:00:00 AM,0031470,"POLYGON ((980694.453 200629.653, 980700.319 20...","[01/28/2021 10:00:00 AM, 01/29/2021 01:00:00 AM]","DatetimeIndex(['2021-01-28 10:00:00', '2021-01...",[1],2021-01-28 10:00:00,307.0,1031703,Financial District-Battery Park City,36061031703
221,559658,01/28/2021 10:00:00 AM,01/29/2021 01:00:00 AM,0234069,"POLYGON ((980788.634 201191.410, 980794.552 20...","[01/28/2021 10:00:00 AM, 01/29/2021 01:00:00 AM]","DatetimeIndex(['2021-01-28 10:00:00', '2021-01...",[1],2021-01-28 10:00:00,307.0,1031703,Financial District-Battery Park City,36061031703
223,559658,01/28/2021 10:00:00 AM,01/29/2021 01:00:00 AM,0031489,None,"[01/28/2021 10:00:00 AM, 01/29/2021 01:00:00 AM]","DatetimeIndex(['2021-01-28 10:00:00', '2021-01...",[1],2021-01-28 10:00:00,NaN,NaN,NaN,NaN
224,559658,01/28/2021 10:00:00 AM,01/29/2021 01:00:00 AM,0031490,"POLYGON ((980912.909 201945.139, 980918.830 20...","[01/28/2021 10:00:00 AM, 01/29/2021 01:00:00 AM]","DatetimeIndex(['2021-01-28 10:00:00', '2021-01...",[1],2021-01-28 10:00:00,40.0,1003900,Tribeca-Civic Center,36061003900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39102,605415,10/09/2021 06:00:00 AM,10/09/2021 03:00:00 PM,0035857,"POLYGON ((991069.555 216546.505, 991074.794 21...","[10/09/2021 06:00:00 AM, 10/09/2021 03:00:00 PM]","DatetimeIndex(['2021-10-09 06:00:00', '2021-10...",[10],2021-10-09 06:00:00,108.0,1010400,Midtown-Times Square,36061010400
39103,605415,10/09/2021 06:00:00 AM,10/09/2021 03:00:00 PM,0184088,"POLYGON ((980896.665 194888.248, 980898.092 19...","[10/09/2021 06:00:00 AM, 10/09/2021 03:00:00 PM]","DatetimeIndex(['2021-10-09 06:00:00', '2021-10...",[10],2021-10-09 06:00:00,7.0,1000900,Financial District-Battery Park City,36061000900
39104,605415,10/09/2021 06:00:00 AM,10/09/2021 03:00:00 PM,0035993,"POLYGON ((990176.336 217036.277, 990179.237 21...","[10/09/2021 06:00:00 AM, 10/09/2021 03:00:00 PM]","DatetimeIndex(['2021-10-09 06:00:00', '2021-10...",[10],2021-10-09 06:00:00,106.0,1010200,Midtown-Times Square,36061010200
39104,605415,10/09/2021 06:00:00 AM,10/09/2021 03:00:00 PM,0035993,"POLYGON ((990176.336 217036.277, 990179.237 21...","[10/09/2021 06:00:00 AM, 10/09/2021 03:00:00 PM]","DatetimeIndex(['2021-10-09 06:00:00', '2021-10...",[10],2021-10-09 06:00:00,108.0,1010400,Midtown-Times Square,36061010400


In [58]:
month1 = data_ct_2021.apply(lambda x: pd.Series(x['month']),axis=1).stack().reset_index(level=1, drop=True)
month1.name = 'month'
data_ct_2021 = data_ct_2021.drop('month', axis=1).join(month1)
data_ct_2021['month'] = data_ct_2021['month'].astype(float).astype(str)
data_ct_2021_event = data_ct_2021[['BoroCT2020','EventID','month']]
data_ct_2021_event = data_ct_2021_event.drop_duplicates()
data_ct_2021_event

,BoroCT2020,EventID,month
214,4094500,559924,1.0
220,1031703,559658,1.0
223,NaN,559658,1.0
224,1003900,559658,1.0
225,1003300,559658,1.0
...,...,...,...
39082,1001300,605415,10.0
39085,1000700,605415,10.0
39102,1010200,605415,10.0
39102,1010400,605415,10.0


# CD33

In [59]:
# Extract the census tract in CD33
cd = gpd.read_file("../vector/Council Districts/nycc.shp")
cd33 = cd[cd['CounDist']==33]

## Count by Event

In [60]:
month_count_2021 = data_ct_2021_event[['BoroCT2020','EventID','month']].groupby(by=['BoroCT2020','month']).count()
month_count_2021 = month_count_2021.rename(columns={"EventID":"Counts"})
# month_count_event = pd.DataFrame(month_count)
month_count_2021.reset_index(inplace = True)
month_count_2021.head()

,BoroCT2020,month,Counts
0,1000201,10.0,1
1,1000201,7.0,2
2,1000201,8.0,1
3,1000202,10.0,3
4,1000202,2.0,1


In [61]:
geo_month_count_2021 = pd.merge(month_count_2021, ct_sel, how="left", on=["BoroCT2020"])
geo_month_count_2021['month'] = geo_month_count_2021['month'].astype(float).astype(int)
geo_month_count_2021.head()

,BoroCT2020,month,Counts,NTAName,GEOID,geometry
0,1000201,10,1,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19..."
1,1000201,7,2,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19..."
2,1000201,8,1,Chinatown-Two Bridges,36061000201,"POLYGON ((988548.218 197770.375, 987978.808 19..."
3,1000202,10,3,Lower East Side,36061000202,"MULTIPOLYGON (((989137.110 196325.439, 989113...."
4,1000202,2,1,Lower East Side,36061000202,"MULTIPOLYGON (((989137.110 196325.439, 989113...."


In [62]:
geo_month_count_2021 = gpd.GeoDataFrame(geo_month_count_2021, crs="EPSG:2263").set_geometry('geometry')
ct_cd33 = gpd.overlay(geo_month_count_2021, cd33, how='intersection')
ct_cd33

,BoroCT2020,month,Counts,NTAName,GEOID,CounDist,Shape_Leng,Shape_Area,geometry
0,1000900,10,3,Financial District-Battery Park City,36061000900,33,132482.563104,1.242545e+08,"POLYGON ((984014.323 192193.577, 984032.883 19..."
1,1000900,11,8,Financial District-Battery Park City,36061000900,33,132482.563104,1.242545e+08,"POLYGON ((984014.323 192193.577, 984032.883 19..."
2,1000900,12,3,Financial District-Battery Park City,36061000900,33,132482.563104,1.242545e+08,"POLYGON ((984014.323 192193.577, 984032.883 19..."
3,1000900,2,1,Financial District-Battery Park City,36061000900,33,132482.563104,1.242545e+08,"POLYGON ((984014.323 192193.577, 984032.883 19..."
4,1000900,3,3,Financial District-Battery Park City,36061000900,33,132482.563104,1.242545e+08,"POLYGON ((984014.323 192193.577, 984032.883 19..."
...,...,...,...,...,...,...,...,...,...
589,3059300,2,2,Greenpoint,36047059300,33,132482.563104,1.242545e+08,"POLYGON ((1002156.099 203750.600, 1001777.645 ..."
590,3059300,4,2,Greenpoint,36047059300,33,132482.563104,1.242545e+08,"POLYGON ((1002156.099 203750.600, 1001777.645 ..."
591,3059300,7,1,Greenpoint,36047059300,33,132482.563104,1.242545e+08,"POLYGON ((1002156.099 203750.600, 1001777.645 ..."
592,3059300,8,1,Greenpoint,36047059300,33,132482.563104,1.242545e+08,"POLYGON ((1002156.099 203750.600, 1001777.645 ..."


In [63]:
pivot_33 = ct_cd33.pivot_table(index =['BoroCT2020'], values =['Counts'],columns='month')
# pivot_33.to_csv('../output/Pivot_permits_cd33_2021_new.csv')
pivot_33

Counts                                                              \
month          1     2     3     4     5     6     7     8     9     10    11   
BoroCT2020                                                                      
1000900       NaN   1.0   3.0   3.0   3.0   NaN   8.0   4.0   4.0   3.0   8.0   
1001501       2.0   3.0   1.0   3.0   4.0   3.0  11.0   7.0   5.0   9.0   8.0   
1001502       8.0  10.0   2.0   7.0  12.0   5.0  18.0  11.0  19.0  16.0   6.0   
1002400       NaN   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN   NaN   
1004400       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   1.0   NaN   
...           ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
3057902      40.0  58.0  83.0  62.0  46.0  25.0  20.0  56.0  45.0  58.0  52.0   
3058901      27.0  49.0  57.0  45.0  27.0  16.0  17.0  32.0  29.0  38.0  45.0   
3058902       NaN   2.0   NaN   1.0   NaN   NaN   NaN   1.0   2.0   NaN   3.0   
3059100       NaN   1.0   1.0   NaN   NaN   1.0   1.0   4.0   2.0   5.0   5.0   
3059300       NaN   2.0   NaN   2.0   NaN   NaN   1.0   1.0   2.0   NaN   2.0   

                  
month         12  
BoroCT2020        
1000900      3.0  
1001501      1.0  
1001502      2.0  
1002400      NaN  
1004400      NaN  
...          ...  
3057902     36.0  
3058901     28.0  
3058902      NaN  
3059100      1.0  
3059300      NaN  

[84 rows x 12 columns]

In [64]:
data_ct_2021_event

,BoroCT2020,EventID,month
214,4094500,559924,1.0
220,1031703,559658,1.0
223,NaN,559658,1.0
224,1003900,559658,1.0
225,1003300,559658,1.0
...,...,...,...
39082,1001300,605415,10.0
39085,1000700,605415,10.0
39102,1010200,605415,10.0
39102,1010400,605415,10.0


## Only 2022_toMarch

In [65]:
data_sel_2022 = data_sel_buffer[data_sel_buffer['StartDateTime']>='2022-1-1']
data_ct_2022 = data_sel_2022.sjoin(ct_sel, how="left")
month1 = data_ct_2022.apply(lambda x: pd.Series(x['month']),axis=1).stack().reset_index(level=1, drop=True)
month1.name = 'month'
data_ct_2022 = data_ct_2022.drop('month', axis=1).join(month1)
data_ct_2022['month'] = data_ct_2022['month'].astype(str)

In [66]:
data_ct_2022

,EventID,StartDateT,EndDateTim,SegmentID,geometry,StartEnd,TimeList,StartDateTime,index_right,BoroCT2020,NTAName,GEOID,month
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"POLYGON ((1000587.646 204453.851, 1000593.600 ...","[03/08/2022 07:00:00 AM, 03/08/2022 07:00:00 PM]","DatetimeIndex(['2022-03-08 07:00:00', '2022-03...",2022-03-08 07:00:00,1208.0,3059100,Greenpoint,36047059100,3.0
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"POLYGON ((1000587.646 204453.851, 1000593.600 ...","[03/08/2022 07:00:00 AM, 03/08/2022 07:00:00 PM]","DatetimeIndex(['2022-03-08 07:00:00', '2022-03...",2022-03-08 07:00:00,1208.0,3059100,Greenpoint,36047059100,3.0
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"POLYGON ((1000587.646 204453.851, 1000593.600 ...","[03/08/2022 07:00:00 AM, 03/08/2022 07:00:00 PM]","DatetimeIndex(['2022-03-08 07:00:00', '2022-03...",2022-03-08 07:00:00,1208.0,3059100,Greenpoint,36047059100,3.0
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"POLYGON ((1000587.646 204453.851, 1000593.600 ...","[03/08/2022 07:00:00 AM, 03/08/2022 07:00:00 PM]","DatetimeIndex(['2022-03-08 07:00:00', '2022-03...",2022-03-08 07:00:00,1208.0,3059100,Greenpoint,36047059100,3.0
0,628327,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,0065898,"POLYGON ((1000587.646 204453.851, 1000593.600 ...","[03/08/2022 07:00:00 AM, 03/08/2022 07:00:00 PM]","DatetimeIndex(['2022-03-08 07:00:00', '2022-03...",2022-03-08 07:00:00,1210.0,3059300,Greenpoint,36047059300,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44833,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023304,"POLYGON ((982367.564 196756.913, 982372.339 19...","[02/04/2022 08:00:00 AM, 02/04/2022 09:30:00 PM]","DatetimeIndex(['2022-02-04 08:00:00', '2022-02...",2022-02-04 08:00:00,5.0,1000700,Financial District-Battery Park City,36061000700,2.0
44833,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023304,"POLYGON ((982367.564 196756.913, 982372.339 19...","[02/04/2022 08:00:00 AM, 02/04/2022 09:30:00 PM]","DatetimeIndex(['2022-02-04 08:00:00', '2022-02...",2022-02-04 08:00:00,15.0,1001502,Financial District-Battery Park City,36061001502,2.0
44833,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023304,"POLYGON ((982367.564 196756.913, 982372.339 19...","[02/04/2022 08:00:00 AM, 02/04/2022 09:30:00 PM]","DatetimeIndex(['2022-02-04 08:00:00', '2022-02...",2022-02-04 08:00:00,15.0,1001502,Financial District-Battery Park City,36061001502,2.0
44834,623949,02/04/2022 08:00:00 AM,02/04/2022 09:30:00 PM,0023303,None,"[02/04/2022 08:00:00 AM, 02/04/2022 09:30:00 PM]","DatetimeIndex(['2022-02-04 08:00:00', '2022-02...",2022-02-04 08:00:00,NaN,NaN,NaN,NaN,2.0


In [67]:
data_ct_2022_event = data_ct_2022[['BoroCT2020','EventID','month']]
data_ct_2022_event = data_ct_2022_event.drop_duplicates()

month_count_2022 = data_ct_2022_event[['BoroCT2020','EventID','month']].groupby(by=['BoroCT2020','month']).count()
month_count_2022 = month_count_2022.rename(columns={"EventID":"Counts"})
# month_count_event = pd.DataFrame(month_count)
month_count_2022.reset_index(inplace = True)
geo_month_count_2022 = pd.merge(month_count_2022, ct_sel, how="left", on=["BoroCT2020"])
geo_month_count_2022['month'] = geo_month_count_2022['month'].astype(float).astype(int)
geo_month_count_2022.head()

geo_month_count_2022 = gpd.GeoDataFrame(geo_month_count_2022, crs="EPSG:2263").set_geometry('geometry')
ct_cd33_2022 = gpd.overlay(geo_month_count_2022, cd33, how='intersection')

In [68]:
ct_cd33_2022['month'].unique()

array([1, 2, 3])

In [69]:
pivot_33_2022 = ct_cd33_2022.pivot_table(index =['BoroCT2020'], values =['Counts'],columns='month')
# pivot_33_2022.to_csv('../output/Pivot_permits_cd33_2022_new.csv')
pivot_33_2022

Counts            
month           1     2     3
BoroCT2020                   
1000900       2.0   3.0   NaN
1001501       NaN   4.0   NaN
1001502       5.0   7.0   NaN
3000100       3.0   3.0   3.0
3000301       2.0   3.0   NaN
...           ...   ...   ...
3057902      63.0  67.0  21.0
3058901      51.0  53.0  20.0
3058902       2.0   3.0   5.0
3059100       3.0   5.0   3.0
3059300       1.0   2.0   5.0

[69 rows x 3 columns]

In [70]:
pivot_33_2022

Counts            
month           1     2     3
BoroCT2020                   
1000900       2.0   3.0   NaN
1001501       NaN   4.0   NaN
1001502       5.0   7.0   NaN
3000100       3.0   3.0   3.0
3000301       2.0   3.0   NaN
...           ...   ...   ...
3057902      63.0  67.0  21.0
3058901      51.0  53.0  20.0
3058902       2.0   3.0   5.0
3059100       3.0   5.0   3.0
3059300       1.0   2.0   5.0

[69 rows x 3 columns]

In [71]:
# data_check= gpd.read_file('../output/filter_seg_3002100.shp') 
# data_check
# data2021_check=data_check[data_check['StartDateT']<'2022-1-1']
# data2021_check=data2021_check[['EventID']]
# data2021_check=data2021_check.drop_duplicates()
# len(data2021_check)

In [72]:
# data2021_check